# NN, ReLU, Xavier, Dropout, and Adam

## NN for MNIST

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [16]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holder
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn layers
W1 = tf.Variable(tf.random_normal([784, 256]))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([256, 256]))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([256, 10]))
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

eval_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(eval_prediction, tf.float32))

In [17]:
# train model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            feed_dict = {X: batch_x, Y: batch_y}
            _cost, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            avg_cost += _cost / total_batch

        print(f'{epoch+1} epoch\ncost: {avg_cost}')
        
    _acc = sess.run(accuracy, feed_dict={
        X: mnist.test.images, Y: mnist.test.labels
    })
    print(f'\naccuracy: {_acc}')

1 epoch
cost: 189.95067534359984
2 epoch
cost: 40.69706348419195
3 epoch
cost: 25.518892918500075
4 epoch
cost: 17.66576872719637
5 epoch
cost: 12.768214062914916
6 epoch
cost: 9.441962210876797
7 epoch
cost: 7.001015768898292
8 epoch
cost: 5.316174130113293
9 epoch
cost: 3.929858521558959
10 epoch
cost: 3.1391922833743067
11 epoch
cost: 2.2395012494217816
12 epoch
cost: 1.7819689082685644
13 epoch
cost: 1.3984012730495883
14 epoch
cost: 1.0590194065964662
15 epoch
cost: 0.9229023197005961

accuracy: 0.9448999762535095


<u>**기본 NN**</u>  <br>
정확도 94%

## Xavier for MNIST

코드가 거의 똑같고, Xavier initialization 부분만 수정하자

In [27]:
# get_variable() 사용 시, 동일한 변수명 겹치면 오류. reset 해줘야 한다.
tf.reset_default_graph()

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holder
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn layers
W1 = tf.get_variable("W1", shape=[784, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[256, 256],
                    initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[256, 10],
                    initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

eval_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(eval_prediction, tf.float32))

In [28]:
# train model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            feed_dict = {X: batch_x, Y: batch_y}
            _cost, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            avg_cost += _cost / total_batch

        print(f'{epoch+1} epoch\ncost: {avg_cost}')
        
    _acc = sess.run(accuracy, feed_dict={
        X: mnist.test.images, Y: mnist.test.labels
    })
    print(f'\naccuracy: {_acc}')

1 epoch
cost: 0.29900466988709845
2 epoch
cost: 0.11244006639515809
3 epoch
cost: 0.07478771491060879
4 epoch
cost: 0.052132528508928716
5 epoch
cost: 0.03782963317099283
6 epoch
cost: 0.03011943416945129
7 epoch
cost: 0.02396332610694859
8 epoch
cost: 0.016440701400173247
9 epoch
cost: 0.016857385337432123
10 epoch
cost: 0.01359818569109351
11 epoch
cost: 0.013620020609232592
12 epoch
cost: 0.010948073213142121
13 epoch
cost: 0.010185245019099317
14 epoch
cost: 0.011250503362018868
15 epoch
cost: 0.008528151728056732

accuracy: 0.9797999858856201


<u>**Xavier initialization**</u><br>
첫 1 epoch 의 cost 도 확 줄어서 시작하는 것을 알 수 있다. (초기값이 잘 설정됨)  <br>
정확도 97% (3% 개선)

## Deep NN for MNIST
NN 을 더 wide 하고 deep 하게 해보자

In [37]:
# get_variable() 사용 시, 동일한 변수명 겹치면 오류. reset 해줘야 한다.
tf.reset_default_graph()

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holder
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn layers
W1 = tf.get_variable("W1", shape=[784, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.get_variable("W4", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

W5 = tf.get_variable("W5", shape=[512, 10],
                    initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

eval_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(eval_prediction, tf.float32))

In [38]:
# train model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            feed_dict = {X: batch_x, Y: batch_y}
            _cost, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            avg_cost += _cost / total_batch

        print(f'{epoch+1} epoch\ncost: {avg_cost}')
        
    _acc = sess.run(accuracy, feed_dict={
        X: mnist.test.images, Y: mnist.test.labels
    })
    print(f'\naccuracy: {_acc}')

1 epoch
cost: 0.2908041570775891
2 epoch
cost: 0.10218646514483477
3 epoch
cost: 0.06963878090747373
4 epoch
cost: 0.05218298674053096
5 epoch
cost: 0.0421375889773481
6 epoch
cost: 0.03556778307178654
7 epoch
cost: 0.02824086396388751
8 epoch
cost: 0.02612629334910625
9 epoch
cost: 0.024941254039506682
10 epoch
cost: 0.01999805359167193
11 epoch
cost: 0.01920121032519754
12 epoch
cost: 0.017522784884310003
13 epoch
cost: 0.015909028814372013
14 epoch
cost: 0.017594474349911748
15 epoch
cost: 0.01457207557789595

accuracy: 0.9782999753952026


<u>**Deep NN + Xavier**</u>  <br>
정확도 97% (이전과 비슷)  <br>
overfitting 이 발생해버렸다..

## Dropout for MNIST
Regularization: Dropout 적용하기

In [39]:
# get_variable() 사용 시, 동일한 변수명 겹치면 오류. reset 해줘야 한다.
tf.reset_default_graph()

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# dropout (keep_prob) rate
# training 일때 0.7, testing 일때 1
keep_prob = tf.placeholder(tf.float32)

# input place holder
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn layers
W1 = tf.get_variable("W1", shape=[784, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[512, 10],
                    initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

eval_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(eval_prediction, tf.float32))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [41]:
# train model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            feed_dict = {X: batch_x, Y: batch_y, keep_prob: 0.7}
            _cost, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            avg_cost += _cost / total_batch

        print(f'{epoch+1} epoch\ncost: {avg_cost}')
        
    _acc = sess.run(accuracy, feed_dict={
        X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1
    })
    print(f'\naccuracy: {_acc}')

1 epoch
cost: 0.4533114428276366
2 epoch
cost: 0.173364374444566
3 epoch
cost: 0.12920449138026352
4 epoch
cost: 0.10840971099720753
5 epoch
cost: 0.09491395228119057
6 epoch
cost: 0.08269532063086948
7 epoch
cost: 0.07465190110304812
8 epoch
cost: 0.06793655089970504
9 epoch
cost: 0.06462515027346931
10 epoch
cost: 0.055708259453742984
11 epoch
cost: 0.056807905252049255
12 epoch
cost: 0.05455706898673353
13 epoch
cost: 0.04926481438652525
14 epoch
cost: 0.0450926262268331
15 epoch
cost: 0.04844479133152749

accuracy: 0.9817000031471252


<u>**Deep NN + Xavier + Dropout**</u>  <br>
정확도 98% (이전에 비해 성능 개선이 이루어짐)